In [1]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
library = 'music_library_202302.xml'

In [3]:
tree = ET.parse(library)
root = tree.getroot()
main_dict=root.findall('dict')
for item in list(main_dict[0]):    
    if item.tag=="dict":
        tracks_dict=item
        break
tracklist=list(tracks_dict.findall('dict'))

print(tracklist[0:5])

[<Element 'dict' at 0x1379395e0>, <Element 'dict' at 0x13795c130>, <Element 'dict' at 0x13795f9f0>, <Element 'dict' at 0x1379635e0>, <Element 'dict' at 0x137965f40>]


In [4]:
podcast=[] #All podcast elements
purchased=[] # All purchased music
apple_music=[] # Music added to lirary through subscription
for item in tracklist:
    x=list(item)
    for i in range(len(x)):
        if x[i].text=="Genre" and x[i+1].text=="Podcast": #          
            podcast.append(list(item))
        if x[i].text=="Kind" and x[i+1].text=="File audio AAC acquistato":
            purchased.append(list(item)) 
        if x[i].text=="Kind" and x[i+1].text=="File audio AAC Apple Music":
            apple_music.append(list(item))

In [5]:
print (f"Number of tracks under Podcast: {str(len(podcast))}")
print (f"Number of tracks Purchased: {str(len(purchased))}")
print (f"Number of Music added thought Apple Music subscription: {str(len(apple_music))}")

Number of tracks under Podcast: 0
Number of tracks Purchased: 27
Number of Music added thought Apple Music subscription: 2720


In [6]:
def cols(kind):
    cols=[]
    for i in range(len(kind)):
        for j in range(len(kind[i])):
            if kind[i][j].tag=="key":
                cols.append(kind[i][j].text)
    return set(cols)

podcast_cols=cols(podcast)
purchased_cols=cols(purchased)
apple_music_cols=cols(apple_music)

In [7]:
print(apple_music_cols)

{'Grouping', 'Sort Name', 'Album Rating Computed', 'Album Artist', 'Year', 'Size', 'Album Rating', 'Disc Count', 'Sample Rate', 'Playlist Only', 'Genre', 'Skip Date', 'Track Type', 'Protected', 'Artwork Count', 'File Folder Count', 'Part Of Gapless Album', 'Date Modified', 'Album', 'Persistent ID', 'Bit Rate', 'Disc Number', 'Location', 'Library Folder Count', 'Sort Album', 'Sort Artist', 'Normalization', 'Sort Composer', 'Track Number', 'Release Date', 'Apple Music', 'Name', 'Play Date UTC', 'Loved', 'Sort Album Artist', 'Play Date', 'Track Count', 'Album Loved', 'Artist', 'Date Added', 'Clean', 'Kind', 'Play Count', 'Composer', 'Total Time', 'Rating', 'Rating Computed', 'Skip Count', 'Track ID', 'Compilation', 'Explicit'}


In [8]:
def df_creation(kind,cols):
    df=pd.DataFrame(columns=cols)
    dict1={}
    for i in range(len(kind)):
        for j in range(len(kind[i])):
            if kind[i][j].tag=="key":
                dict1[kind[i][j].text]=kind[i][j+1].text
        list_values=[i for i in dict1.values()]
        list_keys=[j for j in dict1.keys()]
        df_temp=pd.DataFrame([list_values],columns=list_keys)
        df = pd.concat([df,df_temp],axis=0,ignore_index=True,sort=True)
    return df
df_apple_music = df_creation(apple_music,apple_music_cols)
df_podcast = df_creation(podcast,podcast_cols)
df_purchased = df_creation(purchased,purchased_cols)

In [9]:
df_apple_music.head(10)

,Album,Album Artist,Album Loved,Album Rating,Album Rating Computed,Apple Music,Artist,Artwork Count,Bit Rate,Clean,...,Sort Album Artist,Sort Artist,Sort Composer,Sort Name,Total Time,Track Count,Track ID,Track Number,Track Type,Year
0,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,L'uomo nel mirino,196493,19,773,1,Remote,2004
1,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,Gonfio cosi,219987,19,957,2,Remote,2004
2,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,Solo una botta,226280,19,959,5,Remote,2004
3,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,Momenti no,259667,19,961,6,Remote,2004
4,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,Da questo locale,258093,19,963,7,Remote,2004
5,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,Rap in vena,227573,19,965,8,Remote,2004
6,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,Niente male,249173,19,967,9,Remote,2004
7,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,Faccio sul serio (Radio Version),176080,19,969,19,Remote,2004
8,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,Non crollo,282720,19,971,11,Remote,2004
9,Mr. Simpatia,Fabri Fibra,NaN,100,None,None,Fabri Fibra,1,256,NaN,...,NaN,Fabri Fibra,NaN,Non fare la puttana,194213,19,973,12,Remote,2004


In [10]:
columns_of_interest=['track_id','song_name','play_count','skip_count','album','artist' \
 ,'genre','kind','persistent_id','year_of_release','play_date','skip_date',\
 'release_date','date_modified']
temp_apple = df_apple_music.loc[:,['Track ID','Name','Play Count','Skip Count','Album',\
 'Artist','Genre','Kind','Persistent ID','Year','Play Date UTC',\
 'Skip Date','Release Date','Date Modified']]
temp_purchased = df_purchased.loc[:,['Track ID','Name','Play Count','Skip Count', \
 'Album','Artist','Genre','Kind','Persistent ID','Year',\
 'Play Date UTC','Skip Date','Release Date','Date Modified']]
df_songs = pd.concat([temp_purchased,temp_apple],axis = 0)
df_songs.columns = columns_of_interest

df_songs[['track_id','play_count','skip_count','year_of_release']] = df_songs[['track_id',\
        'play_count','skip_count','year_of_release']].apply(pd.to_numeric)
df_songs[['play_date','skip_date','release_date','date_modified']] = df_songs[['play_date',\
        'skip_date','release_date','date_modified']].apply(pd.to_datetime)

In [11]:
df_songs.sort_values(by='play_count', ascending=False).head(5)

,track_id,song_name,play_count,skip_count,album,artist,genre,kind,persistent_id,year_of_release,play_date,skip_date,release_date,date_modified
22,819,Invece no,134,18,Fenomeno,Fabri Fibra,Hip Hop/Rap,File audio AAC acquistato,B1D84CFA77AC71C2,2017,2022-09-23 21:54:29+00:00,2022-07-06 18:05:41+00:00,2017-04-07 07:00:00+00:00,2017-04-06 21:15:44+00:00
18,811,Dipinto di blu (feat. Laïoung),125,24,Fenomeno,Fabri Fibra,Hip Hop/Rap,File audio AAC acquistato,1B1997CA540DF822,2017,2022-11-27 15:03:28+00:00,2020-09-12 11:47:34+00:00,2017-04-07 07:00:00+00:00,2017-04-06 20:15:44+00:00
1838,4736,REPLAY,110,20,SIRIO,Lazza,Hip-Hop/Rap,File audio AAC Apple Music,AA1A1ED0FC023C92,2022,2023-02-18 13:40:03+00:00,2022-12-21 22:49:36+00:00,2022-04-07 12:00:00+00:00,2022-04-08 05:08:56+00:00
2175,5410,s!r! (feat. Lazza & Sfera Ebbasta),108,15,s!r! (feat. Lazza & Sfera Ebbasta) - Single,thasup,Hip-Hop/Rap,File audio AAC Apple Music,3661EDE1FECE6C80,2022,2023-01-14 16:17:23+00:00,2022-12-26 13:03:30+00:00,2022-07-15 12:00:00+00:00,2022-07-15 05:38:41+00:00
2176,5412,Tití Me Preguntó,108,15,Un Verano Sin Ti,Bad Bunny,Latin,File audio AAC Apple Music,E76C02F2AAF2963E,2022,2023-01-14 16:17:23+00:00,2022-12-26 13:03:30+00:00,2022-05-06 12:00:00+00:00,2022-07-15 05:38:41+00:00
